## Classes

`instance methods` - functions which can be called on objects

`self` - the first argument to all instance methods

#### Classes invariants

Thuths about an object that endure for its lifetime.

Establish in the `__init__` and **raise exceptions** if can't be attempted

#### Implementation detail

Methods with `_` means that represents a implementation method (like a private method, without be private =])

attrs should start with `_`

In [72]:
from pprint import pprint as pp
import sys

In [54]:
class Aircraft:
    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1),
                "ABCDEFGHJK"[:self._num_seats_per_row])
    
    
aircraft = Aircraft("boing", "boing 747", num_rows=23, num_seats_per_row=4)
print(aircraft.registration())
print(aircraft.model())
print(aircraft.seating_plan())

boing
boing 747
(range(1, 24), 'ABCD')


In [108]:
#syntax sugar - self

class Flight:
    def __init__(self, name, aircraft):
        if(len(name) < 5):
            raise ValueError("Name must be, at least, 5 digits")
            
        self._name = name
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [ {letter: None for letter in seats} for _ in rows ]
        
    def name(self):
        return self._name
    
    def aircraft_model(self):
        return self._aircraft.model()
    
    def _parse_seat(self, seat):
        """Parse a seat designator winto a valid row and letter
        
        Args:
            seat: A seat designator succh as 12False
        
        Returns:
            A tuple containing an integer and a string for a row and seat
        """
        
        row_numbers, seat_letters = self._aircraft.seating_plan()
        
        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError("Invalid seat - format NumberLetter".format(letter))
            
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError("Invalid seat row {}".format(row_text))
            
        if row not in row_numbers:
            raise ValueError("Invalid row number {}".format(row))
            
        return row, letter
        
    
    def allocate_seat(self, seat, passenger):
        """Allocate a seat to a passanger
        
        Args:
            seat: A seat designator such as "12C" or "21F",
            passanger: The passenger name
        
        Raises:
            ValueError: If the seat is unavailable
        """
        
        row, letter = self._parse_seat(seat)
        
        if self._seating[row][letter] is not None:
            raise ValueError("Seat {} already occupied".format(seat))
            
        self._seating[row][letter] = passenger
        
    def relocate_passenger(self, from_seat, to_seat):
        """Relocate a passanger to a different seat
        
        Args:
            from_seat: The existing seat designator for the
                       passanger to be moved.
            
            to_seat: The new seat designator.
        """
        from_row, from_letter = self._parse_seat(from_seat)
        if self._seating[from_row][from_letter] is None:
            raise ValueError("No passenger to relocate in seat {}".format(from_seat))
            
        to_row, to_letter = self._parse_seat(to_seat)
        if self._seating[to_row][to_letter] is not None:
            raise ValueError("Seat {} already occupied".format(to_seat))
            
        self._seating[to_row][to_letter] = self._seating[from_row][from_letter]
        self._seating[from_row][from_letter] = None
    
    def num_available_seats(self):
        return sum(sum(1 for s in row.values() if s is None)
                   for row in self._seating
                   if row is not None)
    
    def make_boarding_cards(self, card_printer):
        for passenger, seat in sorted(self._passenger_seats()):
            card_printer(passenger, seat, self.name(), self.aircraft_model())
    
    def _passenger_seats(self):
        """An Iterable series of passenger seating allocations."""
        row_numbers, seat_letters = self._aircraft.seating_plan()
        for row in row_numbers:
            for letter in seat_letters:
                passenger = self._seating[row][letter]
                if passenger is not None:
                    yield (passenger, "{}{}".format(row,letter))

def console_card_printer(passenger, seat, flight_number, aircraft):
    output = "| Name: {0}"     \
             "  Flight: {1}"   \
             "  Seat: {2}"     \
             "  Aircraft: {3}" \
             " |".format(passenger, flight_number, seat, aircraft)
    banner = '+' + '-' *(len(output) - 2) + '+'
    border = '|' + ' ' *(len(output) - 2) + '|'
    lines = [banner, border, output, border, banner]
    card = '\n'.join(lines)
    print(card)
    print()


In [109]:
flight = Flight("latam123", aircraft)

pp(flight.aircraft_model())

try:
    flight.allocate_seat('1C', 'Bruno Almeida')
except NameError as e:
    print("{}".format(str(e)),file=sys.stderr)
    
try:
    flight.allocate_seat('1C', 'Bruno Silveira')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)
       
try:
    flight.allocate_seat('33C', 'Bruno Almeida Silveira')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)
    
try:
    flight.allocate_seat('D1', 'Bruno Almeida Silveira Molina')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)
    
try:
    flight.allocate_seat('7C', 'Suelen Fenali')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)

try:
    flight.relocate_passenger('6C', '9D')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)
    
try:
    flight.relocate_passenger('7C', '1C')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)    
    
try:
    flight.relocate_passenger('7C', '11A')
except BaseException as e:
    print("{}".format(str(e)),file=sys.stderr)

pp(flight._seating)

'boing 747'
[None,
 {'A': None, 'B': None, 'C': 'Bruno Almeida', 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': 'Suelen Fenali', 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': None, 'C': None, 'D': None},
 {'A': None, 'B': N

Seat 1C already occupied
Invalid row number 33
Invalid seat - format NumberLetter
No passenger to relocate in seat 6C
Seat 1C already occupied


In [110]:
flight.make_boarding_cards(console_card_printer)

+----------------------------------------------------------------------+
|                                                                      |
| Name: Bruno Almeida  Flight: latam123  Seat: 1C  Aircraft: boing 747 |
|                                                                      |
+----------------------------------------------------------------------+

+-----------------------------------------------------------------------+
|                                                                       |
| Name: Suelen Fenali  Flight: latam123  Seat: 11A  Aircraft: boing 747 |
|                                                                       |
+-----------------------------------------------------------------------+



In [111]:
# Duck type example

class AirbusA319:
    def __init__(self, registration):
        self._registration = registration
 
    def registration(self):
        return self._registration
    
    def model(self):
        return "Airbus A319"
    
    def seating_plan(self):
        return range(1, 21), "ABCDEF"
    
class Boeing777:
    def __init__(self, registration):
        self._registration = registration
 
    def registration(self):
        return self._registration
    
    def model(self):
        return "Boeing 777"
    
    def seating_plan(self):
        return range(1, 56), "ABCDEFGHK"

## Inheritance

Inheritance in python is most useful for sharing **implementation**

In [117]:
class Aircraft_base:
    def __init__(self, registration):
        self._registration = registration
 
    def registration(self):
        return self._registration
    
    #Cant use this class alone, cause it does not have the implementation for seating_plan method
    def num_seats(self):
        rows, row_seats = self.seating_plan()
        return len(rows) * len(row_seats)
        
class AirbusA319(Aircraft_base):
    def model(self):
        return "Airbus A319"
    
    def seating_plan(self):
        return range(1, 21), "ABCDEF"
    
class Boeing777(Aircraft_base):
    def model(self):
        return "Boeing 777"
    
    def seating_plan(self):
        return range(1, 56), "ABCDEFGHK"
                
airbus = AirbusA319("A123")
boeing = Boeing777("BG27")
                
print("airbus has {} seats".format(airbus.num_seats()))
print("boeing has {} seats".format(boeing.num_seats()))

airbus has 120 seats
boeing has 495 seats
